# Figure 3 (PE‑Young) Analysis Notebook

This notebook generates panels 3B, 3D, and 3F for the PE‑Young sample, loading directly from the raw `.ome.tif` and `.parquet` files under `./data/`. Results are saved to `./results/figure3_PEYoung/`.

## 0. Setup & Imports

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import cv2

from pathlib import Path
from skimage.morphology import remove_small_objects, remove_small_holes, opening, closing, disk
from scipy.ndimage import distance_transform_edt
from spatialdata_io.readers.xenium import xenium, xenium_aligned_image

# Directories
DATA_DIR = Path("./data")
RESULTS_DIR = Path("./results/figure3_PEYoung")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

## 1. Figure 3B: H&E Fiber Segmentation & Overlay

In [ ]:
# Load Xenium data and aligned H&E
sdata = xenium(
    image=DATA_DIR/"morphology_PE-Young.ome.tif",
    matrix=DATA_DIR/"transcripts_PE-Young.parquet",
    cell_boundaries=DATA_DIR/"cell_boundaries_PE-Young.parquet",
    cells=DATA_DIR/"cells_PE-Young.parquet",
    nucleus_boundaries=DATA_DIR/"nucleus_boundaries_PE-Young.parquet"
)
he_img = xenium_aligned_image(
    DATA_DIR/"morphology_PE-Young.ome.tif",
    DATA_DIR/"transcripts_PE-Young.parquet"
)
img = he_img.values.transpose(1,2,0)
img_rot = np.rot90(img, k=1)

# Segment pale-blue fibers
def segment_pale_blue(image, lower=(90,30,40), upper=(130,255,255), min_size=500):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
    mask = opening(mask, disk(3)).astype(bool)
    mask = remove_small_holes(remove_small_objects(mask, min_size), min_size)
    return mask

fibers_mask = segment_pale_blue(img_rot)

# Overlay
overlay = img_rot.copy()
overlay[fibers_mask] = (0,255,255)
blended = cv2.addWeighted(overlay, 0.7, img_rot, 0.3, 0)

plt.figure(figsize=(8,8))
plt.imshow(blended)
plt.axis('off')
plt.title("Figure 3B: PE-Young Fiber Overlay", fontsize=14)
plt.savefig(RESULTS_DIR/"Figure3B_PEYoung_overlay.png", dpi=300, bbox_inches='tight')
plt.show()

## 2. Figure 3D: 100% Stacked Bar of Cell Type by Niche

In [ ]:
# Load precomputed AnnData
adata = sc.read_h5ad(DATA_DIR/"PE-Young.h5ad")

# Compute proportions
niche_groups = ["Fibrotic Niche","Ultraproximal","Proximal","Intermediate","Remote"]
data = {}
for niche in niche_groups + ["All"]:
    if niche=="All":
        ser = adata.obs["cell_type"]
    else:
        ser = adata.obs.loc[adata.obs["cell_niche"]==niche, "cell_type"]
    data[niche] = ser.value_counts(normalize=True)

df = pd.DataFrame(data).T.fillna(0)

# Plot
ax = df.plot(kind='bar', stacked=True, figsize=(10,6), colormap='tab10', edgecolor='k')
ax.set_ylabel("Proportion")
ax.set_xlabel("Spatial Niche")
ax.legend(bbox_to_anchor=(1.02,1), loc='upper left', title="Cell Type")
plt.title("Figure 3D: PE-Young Cell Type Distribution", fontsize=14)
plt.tight_layout()
plt.savefig(RESULTS_DIR/"Figure3D_PEYoung_stacked_bar.png", dpi=300, bbox_inches='tight')
plt.show()

## 3. Figure 3F: Subtype Dotplot by `cell_type2`

In [ ]:
# Define marker genes
marker_genes = {
    "Cardiomyocyte": ["Ryr2","Myl2","Nppb","Myh7"],
    "Fibroblast":    ["Fstl1","Col1a1","Postn"],
    "Endothelial":   ["Cd36","Pecam1","Vwf","Emcn"],
    "Epicardial":    ["Gpx3"],
    "Smooth muscle": ["Myh11","Acta2"],
    "Macrophage":    ["C1qa","Cd68"],
    "Lymphocyte":    ["Ptprc"],
    "Pericyte":      ["Rgs5"],
}

# Dotplot
sc.pl.dotplot(
    adata,
    var_names=marker_genes,
    groupby='cell_type2',
    standard_scale='var',
    figsize=(10,6),
    show=False
)
plt.title("Figure 3F: PE-Young Subtype Dotplot", fontsize=14)
plt.savefig(RESULTS_DIR/"Figure3F_PEYoung_dotplot.png", dpi=300, bbox_inches='tight')
plt.show()